In [1]:
import numpy as np
import pandas as pd

In [2]:
train = pd.read_csv('./data/train.csv')
validation = pd.read_csv('./data/validation_sample.csv')
test = pd.read_csv('./data/test.csv')
submission = pd.read_csv('./data/sample_submission.csv', index_col=0)

train.shape, validation.shape, test.shape, submission.shape

C:\Users\ASIA-11\anaconda3\envs\prj_final\lib\site-packages\numpy\lib\arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


((472972, 3), (3, 1), (1418916, 2), (1418916, 1))

# 전처리

## 마스킹

- 날짜 및 시간 
    - Time Stamp → `<TS>` 
    <br>
    e.g) `2021-02-08T16:21:00` `2021-01-12T07:22:32Z`
    - `<YEAR>` `<DATE>` `<TIME>` `<DAY>`
    <br>
    e.g) `2020 Oct 30 08:39:31` `Fri Sep 18 13:36:11 2020` `2021-01-12T07:22:32Z`


- IP 주소  
    - `127.0.0.1` → `localhost`
    - IP, IP.PORT 패턴 → `<IP>`
    <br>
    e.g) `211.253.243.71` `211.253.243.71.6000`


- 헥사, 시리얼 넘버, 숫자
    - `0x`로 시작하는 16진수 → `<HEX>`
    - 시리얼 넘버 -> `<SN>`
    - `=`, `:`, `'`, ` ` 다음에 오는 16진수 → `<NUM>`
    - 숫자 → `<NUM>`


- 특수문자 처리
    - `][` → `] [`
    - `,` → ` , `
    - 공백 하나로 변환

In [3]:
TARGET_COLUMN = 'full_log'
PATTERNS = [('\d{4}-\d{2}-\d{2}T\d{2}:\d{2}:\d{2}Z?', '<TS>'), 
            # YEAR, MON, DAY, TIME
            ('\d{4}(?= (Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec))', '<YEAR>'), 
            ('(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)\s+\d{1,2}\s+\d{2}:\d{2}:\d{2}', '<DATE> <TIME>'), 
            ('[A-Za-z]{3}\s+<DATE> <TIME>\s+\d{4}', '<DAY> <DATE> <TIME> <YEAR>'), 
            # IP
            ('127.0.0.1', 'localhost'),
            ('\d+\.\d+\.\d+\.\d+(?:\.\d+)?', '<IP>'), 
            # HEX, NUM
            ('(?<![0-9a-fA-F])0x[0-9a-fA-F]+(?=\W|$)', '<HEX>'), 
            ('(?<=\W)(?=[a-fA-F0-9\-]*[0-9])(?=[a-fA-F0-9\-]*[a-fA-F])[a-fA-F0-9]{3,}(?:-[a-fA-F0-9]{3,})+(?=\W|$)', '<SN>'),
            ('(?<==)[a-fA-F0-9]+(?=\W|$)', '<NUM>'),
            ('(?<=:)[a-fA-F0-9]+(?=\s|$)', '<NUM>'),
            ('(?<=\')[a-fA-F0-9]+(?=\')', '<NUM>'),
            ('(?<= )(?=[a-fA-F0-9]*[0-9])[a-fA-F0-9]{4,}(?=\W|$)', '<NUM>'), 
            ('(?<=[^a-zA-Z0-9])(\d+)(?=[^a-zA-Z0-9]|$)', '<NUM>'),
            # 특수문자
            ('\]\[', '] ['), 
            (',', ' , '), 
            ('\s+', ' ')]

def apply_masking(df):
    for pat, repl in PATTERNS:
        df[TARGET_COLUMN] = df[TARGET_COLUMN].str.replace(pat, repl, regex=True)

`train`, `validation`, `test`에 각각 적용

In [4]:
FILES = {'train': train, 'validation': validation, 'test': test}

for file, df in FILES.items():
    apply_masking(df)

In [5]:
train.head()

,id,level,full_log
0,0,0,"<DATE> <TIME> localhost kibana: {""type"":""error..."
1,1,0,"<DATE> <TIME> localhost logstash: [<TS> , <NUM..."
2,2,0,"<DATE> <TIME> localhost kibana: {""type"":""error..."
3,3,0,"<DATE> <TIME> localhost kibana: {""type"":""error..."
4,4,1,type=SYSCALL msg=audit(<NUM>.<NUM>:<NUM>): arc...


## 중복 제거

`full_log`가 동일하지만 `level`이 다른 데이터가 다수 있었습니다. 

따라서 중복되는 데이터 중에서 빈도수가 가장 높은 level의 로그만 남기고 나머지를 제거하였고, 
레벨별 빈도수가 같은 경우에는 level이 낮은 로그만 남기고 나머지를 제거하였습니다.


### 1) 마스킹된 `full_log`를 기준으로 로그가 같은데 위험도 `level`이 다른 경우 추출

1. pivot table로 `full_log`별 `level`의 개수 확인
2. pivot table의 `level` column 중 값이 0 이상인 column의 개수 계산 (→ `count` column 생성)
3. pivot table의 `count` column이 1 보다 큰 로그 데이터 추출

In [6]:
def count_level(df):
    columns = df.columns
    count = 0
    for col in columns:
        count += (df[col] > 0)
    return count

In [7]:
pivot = train.pivot_table(values='id', index='full_log', columns='level', aggfunc='count', fill_value=0)
pivot.head()

level,0,1,2,3,4,5,6
full_log,,,,,,,
"--MARK--: %&)Y , UK%AV1Q$*H_U!C1HyPwEtE?ox8@1xQOHm*%YglknvkO-V$8JeugrDIbo/C.<NUM>^Qn2ZE%rhPbzErRort7-2Dz) , mg%K;?y(MQ5yMC6#Nt2^fJyaRrph!83D1@jawDob9%TooFvx/q*FuL1Vz2'+*Y2qhA/@?8NdvwqDIHWv)7V6H/+R*.f^aw1Lmj9Rh@ir6yU=H#.AQsLG1R?d)Vn1Kh50Xa!?nik , CpaJ;#vF-fU%n]BebMAXe_aC'tn'w-Dnrub_Rv)n#@PlrD8TA , L/'EaJ3oADm?[OBE6;o36b6R4A3f'ECA , zDvQmdH*S#3Zl4Ahd$L;#;w871bsZ#N(HcXzFI[Pr#.B388)w;TIBm5_",1,0,0,0,0,0,0
"--MARK--: 0KY#u2STO^[TWE^uuL.+#a3!+;KeBpHtbuyJ.A&Q_ffa1PSJ]=OuJxVV.A*FuI#*3FBj)/<NUM>'ekCsGF3k!dg$56x%<NUM>'(5stFifpWDtxYnq]%BYiRjz3;w?/)Oh[U!S , ^6Wd-'Iq5*<NUM>$h , _Is0RdURexeSl_O[H!KP;;?#q!BB , Jwtk19MpcPp!kh3nkFAtAlZdGh#-EsgnS9kW5Rd9Vet?TM-ejJpQv;QrD(wnVPK^vDR2D0!^CkI_dq82O?A#6o*=llR026#TolYPZ2p*/Wz@qkTkqD(.@BGT6w1d1FSVCz5+C5ehmz?ndyqC9@&BPa[QmY$vrT2&mmF3tEQ%HQL[J&?OS5+kZ8Du0t$[xNDN4!fl-oSt#gU[$@+0Z;0E'm.I)oZrZ]'",1,0,0,0,0,0,0
"--MARK--: 2k06(D+8h?6q;fFD34mn2lmfGI'(vl6K4Sh##aw39pt_&iORKEzG_K! , , I&@AhO[D59PR3eG(7nFic.S#5P)b@niw , uB*_]n!&.fiy.aS*3st[/iLcFDn3&Y*$] , W!FKh&i!Mwxs&zNJUIhyzLxCJ+*]RPMAzC5BKv7s/GM23J**owGveQJeKj1Uq?rHD=l88zP_.wUVrSCG'9w[DW2GeR-u-@1C4 , -DdeRroR8^_$eHMS$ElzEmdZb$c-IqIu2N.]?yJnu(T)aE(4Kc9gP!VR9Y!'o).jM= , gkvOryu_vB)d5Oo4'3ctt?L[<NUM>'El6#fxla_RGK;!t^sV&_OH9$=<NUM>[Cl-[G7X)E]Q!UyvSd-h51@uAj//ZJfm/Y4PZRICu&o*u , %",1,0,0,0,0,0,0
"--MARK--: =?s_w*?!5u]7c/s?Z1(FATAl?o0khglYB]Q0OTwjGzB!nVlVSdiJnnN/VJJ#qi091Xao?iduzHEdkUw&rDtQQc2sV7(6a8$U#j0MC&wG , YRG$rbf+@1c0-n6E^.2Cv&<NUM>=T7mXZ$D]1nf9yl6@QER690Yup*5Zt48Lu'9gc65Ixx]B2'JMqv_qAY0=XEdVmW6NpRRFqy+3S&mKFPru+erp3d]mRJ;v@D*8nfRlrFu!1vU;pNsHl5524*pD)VOkSOf$<NUM>(&)J?E6FERs4KV;'y6cZxVcO6Rf/@+s?kK5LSVHo'D2](^l_zzD9]5o&m!&;Kw^@*.?tBL. , L.K))IKkU*6kD4+DmgYrYG9$WZn)hTf0ozna4wdJ&f5u]te.<NUM>&)vSJdMUuNU55H??J3LNru!T@PZja(b5h.atdRP6T/Kq5A5M)!dIL36c0h-_TPg3*%5d*+(*5H!?EU3_c/wEYGq",1,0,0,0,0,0,0
"--MARK--: Bve5Z-oLF9#)LT;.snZlXJbizTDVmEOjQP86?i5;[k0Ir(w=u^M#G5PZVNtzL2N/aIA16bz;f6^(e962LkZjt_gIx(w]/sAoxENMgi3!!ql?&;yz53$'q#+#Hf_g$W/c9TR)jpB , 4z_zc40reDUAx6xn?i*3nPu[01FZg)oGqKoq%<NUM>;3RKKw9zuFNpsjDqj_35c?VfE^vVE27MQ[<NUM>-PL/'Hl]<NUM>!];yKVNZWJH!i(pv$1AWrT$@[m5]Y=?]b/yRu17FroqeFrTg2!ErRoR-pJW5oqPotFmPyj*D&WS&sR% , YVsbS70HCHFjcynWpmYPW+@YTvO-$]&9s67rxW",1,0,0,0,0,0,0


In [8]:
pivot['count'] = count_level(pivot)
pivot.head()

level,0,1,2,3,4,5,6,count
full_log,,,,,,,,
"--MARK--: %&)Y , UK%AV1Q$*H_U!C1HyPwEtE?ox8@1xQOHm*%YglknvkO-V$8JeugrDIbo/C.<NUM>^Qn2ZE%rhPbzErRort7-2Dz) , mg%K;?y(MQ5yMC6#Nt2^fJyaRrph!83D1@jawDob9%TooFvx/q*FuL1Vz2'+*Y2qhA/@?8NdvwqDIHWv)7V6H/+R*.f^aw1Lmj9Rh@ir6yU=H#.AQsLG1R?d)Vn1Kh50Xa!?nik , CpaJ;#vF-fU%n]BebMAXe_aC'tn'w-Dnrub_Rv)n#@PlrD8TA , L/'EaJ3oADm?[OBE6;o36b6R4A3f'ECA , zDvQmdH*S#3Zl4Ahd$L;#;w871bsZ#N(HcXzFI[Pr#.B388)w;TIBm5_",1,0,0,0,0,0,0,1
"--MARK--: 0KY#u2STO^[TWE^uuL.+#a3!+;KeBpHtbuyJ.A&Q_ffa1PSJ]=OuJxVV.A*FuI#*3FBj)/<NUM>'ekCsGF3k!dg$56x%<NUM>'(5stFifpWDtxYnq]%BYiRjz3;w?/)Oh[U!S , ^6Wd-'Iq5*<NUM>$h , _Is0RdURexeSl_O[H!KP;;?#q!BB , Jwtk19MpcPp!kh3nkFAtAlZdGh#-EsgnS9kW5Rd9Vet?TM-ejJpQv;QrD(wnVPK^vDR2D0!^CkI_dq82O?A#6o*=llR026#TolYPZ2p*/Wz@qkTkqD(.@BGT6w1d1FSVCz5+C5ehmz?ndyqC9@&BPa[QmY$vrT2&mmF3tEQ%HQL[J&?OS5+kZ8Du0t$[xNDN4!fl-oSt#gU[$@+0Z;0E'm.I)oZrZ]'",1,0,0,0,0,0,0,1
"--MARK--: 2k06(D+8h?6q;fFD34mn2lmfGI'(vl6K4Sh##aw39pt_&iORKEzG_K! , , I&@AhO[D59PR3eG(7nFic.S#5P)b@niw , uB*_]n!&.fiy.aS*3st[/iLcFDn3&Y*$] , W!FKh&i!Mwxs&zNJUIhyzLxCJ+*]RPMAzC5BKv7s/GM23J**owGveQJeKj1Uq?rHD=l88zP_.wUVrSCG'9w[DW2GeR-u-@1C4 , -DdeRroR8^_$eHMS$ElzEmdZb$c-IqIu2N.]?yJnu(T)aE(4Kc9gP!VR9Y!'o).jM= , gkvOryu_vB)d5Oo4'3ctt?L[<NUM>'El6#fxla_RGK;!t^sV&_OH9$=<NUM>[Cl-[G7X)E]Q!UyvSd-h51@uAj//ZJfm/Y4PZRICu&o*u , %",1,0,0,0,0,0,0,1
"--MARK--: =?s_w*?!5u]7c/s?Z1(FATAl?o0khglYB]Q0OTwjGzB!nVlVSdiJnnN/VJJ#qi091Xao?iduzHEdkUw&rDtQQc2sV7(6a8$U#j0MC&wG , YRG$rbf+@1c0-n6E^.2Cv&<NUM>=T7mXZ$D]1nf9yl6@QER690Yup*5Zt48Lu'9gc65Ixx]B2'JMqv_qAY0=XEdVmW6NpRRFqy+3S&mKFPru+erp3d]mRJ;v@D*8nfRlrFu!1vU;pNsHl5524*pD)VOkSOf$<NUM>(&)J?E6FERs4KV;'y6cZxVcO6Rf/@+s?kK5LSVHo'D2](^l_zzD9]5o&m!&;Kw^@*.?tBL. , L.K))IKkU*6kD4+DmgYrYG9$WZn)hTf0ozna4wdJ&f5u]te.<NUM>&)vSJdMUuNU55H??J3LNru!T@PZja(b5h.atdRP6T/Kq5A5M)!dIL36c0h-_TPg3*%5d*+(*5H!?EU3_c/wEYGq",1,0,0,0,0,0,0,1
"--MARK--: Bve5Z-oLF9#)LT;.snZlXJbizTDVmEOjQP86?i5;[k0Ir(w=u^M#G5PZVNtzL2N/aIA16bz;f6^(e962LkZjt_gIx(w]/sAoxENMgi3!!ql?&;yz53$'q#+#Hf_g$W/c9TR)jpB , 4z_zc40reDUAx6xn?i*3nPu[01FZg)oGqKoq%<NUM>;3RKKw9zuFNpsjDqj_35c?VfE^vVE27MQ[<NUM>-PL/'Hl]<NUM>!];yKVNZWJH!i(pv$1AWrT$@[m5]Y=?]b/yRu17FroqeFrTg2!ErRoR-pJW5oqPotFmPyj*D&WS&sR% , YVsbS70HCHFjcynWpmYPW+@YTvO-$]&9s67rxW",1,0,0,0,0,0,0,1


In [9]:
pivot[pivot['count'] > 1]

level,0,1,2,3,4,5,6,count
full_log,,,,,,,,
<DATE> <TIME> localhost sudo: apache : TTY=unknown ; PWD=/var/www/html/management ; USER=root ; COMMAND=/bin/curl -XGET localhost:<NUM>/_cat/snapshots/esild_backup,0,4,1,0,0,0,0,2
juniper,4,0,0,1,0,0,0,2
"level : %{rule.level} , log : %{log}",10,8,0,0,0,0,0,2
"level : <NUM> , log : 'commit at' was successful",1,1,0,0,0,0,0,2
"level : <NUM> , log : Aborting , unable to run in the background as a daemon: error-message",1,4,0,0,0,0,0,2
...,...,...,...,...,...,...,...,...
"level : <NUM> , log : task-name: lost interface-hierarchy logical-interface-index for route route-prefix",1,1,0,0,0,0,0,2
"level : <NUM> , log : traceRouteCtlOwnerIndex = test-owner , traceRouteCtlTestName = test-name",2,1,0,0,0,0,0,2
"level : <NUM> , log : type username is invalid",1,1,0,0,0,0,0,2


### 2) 다수의 `level` 을 갖는 로그마다 하나의 `level` 선택

- 빈도수가 가장 높은 level 선택
- 레벨별 빈도수가 같은 경우에는 낮은 level 선택

→ 선택한 level로 `level` column 생성

In [10]:
dupl = pivot[pivot['count'] > 1].copy()
dupl['level'] = np.argmax(dupl.iloc[:, :-1].values, axis=-1)
dupl

level,0,1,2,3,4,5,6,count,level
full_log,,,,,,,,,
<DATE> <TIME> localhost sudo: apache : TTY=unknown ; PWD=/var/www/html/management ; USER=root ; COMMAND=/bin/curl -XGET localhost:<NUM>/_cat/snapshots/esild_backup,0,4,1,0,0,0,0,2,1
juniper,4,0,0,1,0,0,0,2,0
"level : %{rule.level} , log : %{log}",10,8,0,0,0,0,0,2,0
"level : <NUM> , log : 'commit at' was successful",1,1,0,0,0,0,0,2,0
"level : <NUM> , log : Aborting , unable to run in the background as a daemon: error-message",1,4,0,0,0,0,0,2,1
...,...,...,...,...,...,...,...,...,...
"level : <NUM> , log : task-name: lost interface-hierarchy logical-interface-index for route route-prefix",1,1,0,0,0,0,0,2,0
"level : <NUM> , log : traceRouteCtlOwnerIndex = test-owner , traceRouteCtlTestName = test-name",2,1,0,0,0,0,0,2,0
"level : <NUM> , log : type username is invalid",1,1,0,0,0,0,0,2,0


### 3) `level` column에서 지정한 level이 아닌 로그 제거

In [11]:
print(train.shape)
train['level'].value_counts()

(472972, 3)


0    334065
1    132517
3      4141
5      2219
2        12
4        10
6         8
Name: level, dtype: int64

In [12]:
for index in dupl.index:
    targets = train[(train['full_log']==index) & (train['level']!=dupl['level'][index])].index
    train.drop(index=targets, inplace=True)
    
print(train.shape)
train['level'].value_counts()

(472550, 3)


0    334020
1    132182
3      4139
5      2180
2        11
4        10
6         8
Name: level, dtype: int64